In [48]:
import os
import tensorflow as tf
import numpy as np
import json

SEED = 1234
tf.random.set_seed(SEED)

In [49]:
#Creation of data generator objects

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                       width_shift_range=10,
                                       height_shift_range=10,
                                       zoom_range=0.3,
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       fill_mode='constant',#when we flip, rotate, we introduce new pixels and we must set a way on how to fill
                                       cval=0,
                                       rescale=1/255.)

else:
    train_data_gen = ImageDataGenerator(rescale=1/255.)
    
valid_data_gen = ImageDataGenerator(rescale=1/255.)


In [50]:
#Setting paths for training, validation and test set

cwd = os.path.join(os.path.dirname(os.getcwd()),"artificial-neural-networks-and-deep-learning-2020/MaskDataset/")

dataset_dir = os.path.join(cwd, "training/")
validation_dir = os.path.join(cwd, "validation/")
test_dir = os.path.join(cwd, "test/")

path = os.getcwd()
print ("The current working directory is %s" % test_dir)

with open(os.path.join(cwd,'train_gt.json')) as f:
  myJson = json.load(f)
print(myJson)

#division of images in different set

for key in myJson:
    os.rename(os.path.join(dataset_dir,key), dataset_dir+'/'+str(myJson[key])+'/'+key)

In [ ]:
#reset validation set to empty set
for i in range(3):
    curr_dir = os.path.join(cwd, "validation/" + str(i))
    image_filenames = next(os.walk(curr_dir))[2]

    for image_name in image_filenames:
        os.rename(os.path.join(curr_dir,image_name), dataset_dir+'/'+str(myJson[image_name])+'/'+image_name)

In [53]:
#choose 5% random images from the training set and move them to validation set
random.seed(SEED)

for i in range(3):
    curr_dir = os.path.join(cwd, "training/" + str(i))
    image_filenames = next(os.walk(curr_dir))[2]

    for image_name in image_filenames:
        if 100*random.random() < 5:
            os.rename(os.path.join(curr_dir,image_name), validation_dir+'/'+str(myJson[image_name])+'/'+image_name)
    


In [54]:
#Generators for reading images dirrectly from directories

img_h = 408
img_w = 612

bs=16

#training generator
train_gen = train_data_gen.flow_from_directory(dataset_dir,
                                               target_size=(img_h,img_w),
                                              color_mode='rgb',
                                              batch_size=bs,
                                              class_mode='categorical',
                                              shuffle=True,
                                              seed=SEED)
#validation generator
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               target_size=(img_h,img_w),
                                               color_mode='rgb',
                                              batch_size=bs,
                                              class_mode='categorical',
                                              shuffle=True,
                                              seed=SEED)


Found 5348 images belonging to 3 classes.
Found 266 images belonging to 3 classes.


In [ ]:
#creation of dataset objects
num_classes = 3


train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                              output_types=(tf.float32,tf.float32),
                                              output_shapes=([None,img_h,img_w,3],[None,num_classes]))

train_dataset=train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                              output_types=(tf.float32,tf.float32),
                                              output_shapes=([None,img_h,img_w,3],[None,num_classes]))

valid_dataset=valid_dataset.repeat()

In [ ]:
import matplotlib.pyplot as plt

iterator = iter(train_dataset)


In [ ]:
augmented_img, target = next(iterator)
augmented_img = np.array(augmented_img[0])
augmented_img = augmented_img * 255

plt.imshow(np.uint8(augmented_img))
plt.plot

In [ ]:
start_f = 9
depth = 6

model = tf.keras.Sequential()
for i in range(depth):
    if i == 0:
        input_shape = [img_h,img_w,3]
    else:
        input_shape = [None]
    
    model.add(tf.keras.layers.Conv2D(filters=start_f,
                                    kernel_size=(3,3),
                                    strides=(1,1),
                                    padding='same',
                                    input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
    start_f *=2
    
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
model.add(tf.keras.layers.Dense(units=512, activation='relu'))

model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
model.summary()

In [ ]:
#Loss function
loss = tf.keras.losses.CategoricalCrossentropy()

#Learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

#Validation metric used during training
metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss , metrics=metrics)

In [ ]:
#Callbacks list

from datetime import datetime

exps_dir = os.path.join(cwd, 'classification_experiments_')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d')

exp_name='FC'

exp_dir = os.path.join(exps_dir, exp_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

#Checkpoint callback
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir,'cp.ckpt'),save_weights_only=True)

callbacks.append(ckpt_callback)

#tensorboard callback
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
tb_callback=tf.keras.callbacks.TensorBoard(log_dir=tb_dir,histogram_freq=1)


# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)



In [ ]:
#model.load_weights(filepath=os.path.join(cwd,'cp.ckpt'))


model.fit(x=train_dataset,
         epochs=100,
         steps_per_epoch=len(train_gen),
        validation_data=valid_dataset,
         validation_steps=len(valid_gen),
         callbacks=callbacks)

In [ ]:
#Function for creating the result in a csv file

from datetime import datetime

def create_csv(results, results_dir='./results'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')




In [ ]:
#Prediction computation done one image at time

from PIL import Image

image_filenames = next(os.walk(test_dir))[2]

results = {}
for image_name in image_filenames:
    
    image = tf.keras.preprocessing.image.load_img(test_dir+image_name,target_size=(img_h,img_w))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    
    predictions = model.predict_classes(input_arr*(1/255))
    #img = Image.open(test_dir+'/'+image_name).convert('RGB')
    #img_array = np.array(img)
    #img_array = np.expand_dims(img_array, 0) 
    #data_normalization
    #print(np.argmax(predictions, axis=1)[0] )

    #prediction = argmax(softmax)   # predicted class
    

    #results[image_name] = np.argmax(predictions, axis=1) [0]
    
    results[image_name] = predictions[0]

create_csv(results)
print (results)

In [ ]:
model.save_weights(filepath=os.path.join(cwd,'cp.ckpt'))

In [ ]:
print(cwd)

In [ ]:
image = tf.keras.preprocessing.image.load_img(test_dir+image_name,target_size=(img_h,img_w))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])*1/255
print(input_arr.shape)

#augmented_img = augmented_img * 255
print (input_arr)

plt.imshow(np.uint8(input_arr*255)[0])
plt.plot